Registration information:

- [The relationship between sleep and academic performance in first-year college students: a longitudinal, multi-university analysis](https://osf.io/5xngv?view_only=bf5d479c7e4a409dbd72b15165bfd560)
- [The relationship between sleep and academic performance in first-year college students: a longitudinal, multi-university analysis (NetHealth continuation)](https://osf.io/x76b4?view_only=3d8f8c27841d4a5e90fce711c936c0ed)

In [1]:
import os
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyfit
import csv, json, sys
import math
import matplotlib.pyplot as plt
import matplotlib.dates as dates

from shutil import copyfile
from pandas.io.json import json_normalize
from tqdm import tqdm
from datetime import datetime, timedelta, time, date
from sklearn.linear_model import LinearRegression
from scipy import stats
from statistics import median
import statsmodels.api as sm
import warnings
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.transforms import offset_copy
import seaborn as sns


#warnings.simplefilter("ignore")


**Start Of UW-Specific Work**

The raw UWEXP Fitbit data was provided as JSON files. We converted these to csv files in the same format as the Life@CMU datasets to run this data through the same pipeline. The following section converts the raw UWEXP data (given in "Daily Sleep/") to "sleep_raw_data/" and "steps_raw_data/"

In [ ]:
# copy all json files from raw data to json folder
def copyAllJsonFilesFromRawToJsonFolder(phase):

    if phase == 'uw1':
        uw = 'Fitbit UWEXPI/'
    elif phase == 'uw2':
        uw = 'Fitbit UWEXPII/'
    else:
        raise('Invalid phase argument')

    raw_dir = os.path.join(uw, 'Daily Sleep')
    csv_dir = os.path.join(uw, 'daily_sleep_raw_csv')
    json_dir = os.path.join(uw, 'daily_sleep_raw_json')

    # create directories if they don't yet exist
    dir_list = [csv_dir, json_dir]
    for d in dir_list:
        if not os.path.exists(d): 
            os.makedirs(d)

    for f in os.listdir(raw_dir):
        if f[-4:] == 'json': 
            copyfile(os.path.join(raw_dir,f),
                     os.path.join(json_dir,f))

In [ ]:
# UNCOMMENT TO RUN
copyAllJsonFilesFromRawToJsonFolder('uw1')
copyAllJsonFilesFromRawToJsonFolder('uw2')

In [ ]:
# Converts one section of sleep episode data to a pandas dataframe
# input: json_data[0]['sleep'][0] = one sleep episode data in JSON
# output: dataframe with a dateTime = '%Y-%m-%d %H:%M:%S' and 'value' column
def getDF(json_sleep_episode):
    
    # get the JSON values as dataframe
    json_df = json_normalize(json_sleep_episode['minuteData']) # YSS: could just use pd.DataFrame(json_sleep_episode['minuteData'])
    
    # create a minute index with all the minutes in the sleep episode
    start = json_sleep_episode['startTime']
    end = json_sleep_episode['endTime']
    TIME_INDEX = pd.date_range(start=start,end=end,freq='min')[:-1] # [:-1] b/c json excludes end point
    
    #YSS
    if len(TIME_INDEX) != json_df.shape[0]:
        # it is expected that there is a one-to-one matching between dateTime values in minuteData 
        # and the time index generated between start and end times
        print('\t\tinvestigate mismatch in timing of sleep data')
    
    # build final dataframe
    df = pd.DataFrame(index=TIME_INDEX)
    df['dt'] = df.index.strftime('%Y-%m-%d %H:%M:%S')
    df['time'] = df['dt'].apply(lambda x: x.split()[1])
    df = df.merge(json_df, how='left', left_on='time', right_on='dateTime')
    df = df[['dt', 'value']]
    df = df.rename(columns={'dt':'dateTime'})
    
    #YSS
    if len(TIME_INDEX) != df.shape[0]:
        # it is expected that there is a one-to-one matching between dateTime values in minuteData 
        # and the time index generated between start and end times
        print('\t\tinvestigate issue in timestamping sleep data')
    
    return df

In [ ]:
# input: json file
# output: raw sleep dataframe
def getParticipantSleepData(filename):

    with open(filename, 'r') as f:
        data = f.readlines()

    # fix JSON formatting errors
    count = data[0].count("}}") - 1
    data_val = '[' + data[0].replace('}}','}},', count) + ']'

    # load JSON data
    json_data = json.loads(data_val)
    
    # initialize empty dataframe
    all_df = pd.DataFrame()
    #all_df = [] #YSS
    
    # append all sleep episodes to empty dataframe
    for idx, sleep_day in enumerate(json_data):
        print('\tprocessing sleep day {:03}...'.format(idx)) #YSS
        for episode in sleep_day['sleep']:
            episode_df = getDF(episode)
            all_df = all_df.append(episode_df) #YSS use of concatenation is more efficient
            
    return all_df
    #return pd.concat(all_df) #YSS

In [ ]:
def convertAllJSONToCSV(phase):
    
    print('Beginning directory conversion...')

    if phase == 'uw1':
        uw = 'Fitbit UWEXPI/'
    elif phase == 'uw2':
        uw = 'Fitbit UWEXPII/'
    else:
        raise('Invalid phase argument')

    csv_dir = os.path.join(uw, 'daily_sleep_raw_csv') #YSS sleep_raw_csv -> daily_sleep_raw_csv
    json_dir = os.path.join(uw, 'daily_sleep_raw_json')        
        
    for fn in tqdm(os.listdir(json_dir)):
        print('converting {}...'.format(fn)) #YSS
        df = getParticipantSleepData(os.path.join(json_dir, fn))
        new_fn = os.path.join(csv_dir,fn.replace('.json', '.csv'))
        df.to_csv(new_fn, index=False)

In [ ]:
# UNCOMMENT TO RUN
convertAllJSONToCSV('uw1')
convertAllJSONToCSV('uw2')
#YSS no further investigation was reported (good!)

In [ ]:
# Convert steps csv to correct format
# columns: datetime (%Y-%m-%d %H:%M:%S) and steps
def processStepsFile(src, dest):
    df = pd.read_csv(src)
    df['datetime'] = df.date.str.cat(df.time, sep=" ")
    new_df = df[['datetime', 'steps']]
    new_df.to_csv(dest, index=False)

In [ ]:
# Get entire steps raw folder
def processAllSteps(phase):
    
    print('Beginning directory conversion...')
    
    if phase == 'uw1':
        uw = 'Fitbit UWEXPI/'
    elif phase == 'uw2':
        uw = 'Fitbit UWEXPII/'
    else:
        raise('Invalid phase argument')


    steps_all_dir = os.path.join(uw, 'Daily Step Details')
    steps_dir = os.path.join(uw, 'daily_steps_raw_data') #YSS steps_raw_data -> daily_steps_raw_data
    
    if not os.path.exists(steps_dir): 
        os.makedirs(steps_dir)    
    
    for fn in tqdm(os.listdir(steps_all_dir)):
        if fn[-3:] == 'csv': 
            src = os.path.join(steps_all_dir, fn)
            dest = os.path.join(steps_dir, fn)
            processStepsFile(src, dest)

In [ ]:
# UNCOMMENT TO RUN
processAllSteps('uw1')
processAllSteps('uw2')
#YSS NOTE use of detailed step data is potentially problematic because of missing data otherwise available in daily step data

In [ ]:
# renames files from 'PID001_step.csv' -> 'PID001.csv'
def convertAllFilenamesToID(phase):
    if phase == 'uw1':
        uw = 'Fitbit UWEXPI/'
    elif phase == 'uw2':
        uw = 'Fitbit UWEXPII/'
    else:
        raise('Invalid phase argument')
    
    sleep_dir = os.path.join(uw,'daily_sleep_raw_csv/') #YSS sleep_raw_data --> daily_sleep_raw_csv
    steps_dir = os.path.join(uw,'daily_steps_raw_data/') #YSS steps_raw_data -> daily_steps_raw_data
    
    sleep_files = os.listdir(sleep_dir)
    steps_files = os.listdir(steps_dir)
    
    #YSS
    sleep_fns_unique = set([fn[:6] for fn in sleep_files])
    if len(sleep_fns_unique) != len(sleep_files):
        print('investigate split sleep data being overwritten')
    step_fns_unique = set([fn[:6] for fn in steps_files])
    if len(step_fns_unique) != len(steps_files):
        print('investigate split step data being overwritten')
    
    for f in sleep_files:
        src = os.path.join(sleep_dir,f)
        new_f = f[:6] +'.csv'
        dest = os.path.join(sleep_dir, new_f)
        os.rename(src,dest)
        
    for f in steps_files:
        src = os.path.join(steps_dir,f)
        new_f = f[:6] +'.csv'
        dest = os.path.join(steps_dir, new_f)
        os.rename(src,dest)

In [ ]:
# UNCOMMENT TO RUN
convertAllFilenamesToID('uw1')
convertAllFilenamesToID('uw2')
#YSS PID136 data is split and the second one overwrited the first one in both sleep and step data

**End Of UW-Specific Work**

We have multiple datasets, and getPath allows us to grab the path of whatever dataset we're working with. Each dataset folder assumes a similar underlying directory structure:
* steps_raw_data
* sleep_raw_data
* sleep_steps_data
* sleep_episodes
* computed_features

The steps_raw_data is optional, and for the NetHealth dataset, sleep_steps_data is missing steps. We don't need steps to help us compute sleep, rather steps is useful in differentiating missing data from all-nighters.

In [ ]:
def getPath(phase):
    if phase == 'uw1':
        path = 'Fitbit UWEXPI/'    
    elif phase == 'uw2':
        path = 'Fitbit UWEXPII/'
    elif phase == 'lac1':
        path = 'LifeAtCMU_Phase1/'
    elif phase == 'lac2':
        path = 'LifeAtCMU_Phase2/'
    elif phase == 'nh':
        path = 'NetHealth/'
    else:
        raise('Invalid phase argument')
    return path

In [ ]:
def createSleepStepsDir():
    for d in ['uw1','uw2','lac1','lac2']:
        new_d = os.path.join(getPath(d),'sleep_steps_data/')
        if not os.path.exists(new_d): 
            os.makedirs(new_d)

In [ ]:
# UNCOMMENT TO RUN
createSleepStepsDir()

The following combines all sleep and steps into a single combined dataframe for the given date range in the sleep_steps_data folder. This step is optional for these sleep analyses, and is done for NetHealth in the NetHealth cleaning code.

In [ ]:
# combines all raw sleep and raw steps 
# input: "uw1", "uw2", "lac1", "lac2"
def saveAllCombined(phase):

    if phase == 'uw1':
        TIME_INDEX = pd.date_range(start='1/1/2018',
                                   end='6/13/2018',
                                   freq='min')
    elif phase == 'uw2': 
        TIME_INDEX = pd.date_range(start='4/1/2019',
                                   end='6/14/2019',
                                   freq='min')
    elif phase == 'lac1':
        TIME_INDEX = pd.date_range(start='1/16/2017',
                                   end='5/15/2017',
                                   freq='min')
    elif phase == 'lac2':
        TIME_INDEX = pd.date_range(start='1/16/2018',
                                   end='5/15/2018',
                                   freq='min')
    else:
        raise('Invalid phase argument')
        
    path = getPath(phase)
    
    
    
    sleep_dir = os.path.join(path,'daily_sleep_raw_csv/') #YSS sleep_raw_data --> daily_sleep_raw_data
    steps_dir = os.path.join(path,'daily_steps_raw_data/') #YSS steps_raw_data -> daily_steps_raw_data
    combined_dir = os.path.join(path,'sleep_steps_data/')

    sleep_files = os.listdir(sleep_dir)
    steps_files = os.listdir(steps_dir)
    both = list(set(sleep_files) & set(steps_files))
    both.sort() #YSS

    DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
    
    bad_ids = []
    for idx in tqdm(range(len(both))):
        
        #print('check if {} exists ...'.format(os.path.join(combined_dir, both[idx])), end=' ') #YSS

        if os.path.exists(os.path.join(combined_dir, both[idx])):
            #print('YES; continue to the next file') #YSS
            continue
        else:
            #print('NO; combine sleep and step files') #YSS
            try:
                df_sleep = pd.read_csv(os.path.join(sleep_dir, both[idx]))
                df_steps = pd.read_csv(os.path.join(steps_dir, both[idx]))

                #YSS - check if all rows have consistent second values
                secs = df_sleep['dateTime'].apply(lambda time: time[-2:]).unique()
                if len(secs) > 1:
                    print('inconsistent sleep second values for {}: {}'.format(both[idx], list(secs)))
                secs = df_steps['datetime'].apply(lambda time: time[-2:]).unique()
                if len(secs) > 1:
                    print('inconsistent step second values for {}: {}'.format(both[idx], list(secs)))
                # NOTE should there be inconsistentsies, there is a chance that replacing seconds with 0 results in 
                #      duplicate timestamps
                #print('finished 1st check')


                # Convert times to datetime objects
                df_sleep['dt_typed'] = df_sleep['dateTime'].apply(lambda time: datetime.strptime(time,DATE_FORMAT).replace(second=0))
                df_steps['dt_typed'] = df_steps['datetime'].apply(lambda time: datetime.strptime(time,DATE_FORMAT).replace(second=0))
                df_sleep = df_sleep.rename(columns={'value' : 'sleep_value'})

                #YSS - check if any rows have date&time values that do not appear in TIME_INDEX
                dt_index = df_sleep['dt_typed'].isin(TIME_INDEX)
                if (~dt_index).sum() > 0:
                    print('inconsistent sleep timestamps for {}: {}'.format(both[idx], df_sleep[~dt_index]['dt_typed']))
                dt_index = df_steps['dt_typed'].isin(TIME_INDEX)
                if (~dt_index).sum() > 0:
                    print('inconsistent step timestamps for {}: {}'.format(both[idx], df_steps[~dt_index]['dt_typed']))
                # NOTE if all timestamps exist in TIME_INDEX there should not be any duplicates
                #print('finished 2nd check')

                #YSS - check if there are duplicate timestamps
                duplicates = df_sleep.groupby(by=['dt_typed']).size()
                if duplicates[duplicates > 1].shape[0] > 0:
                    print('duplicates in sleep dataframe for {}: {}'.format(both[idx], duplicates[duplicates > 1]))
                duplicates = df_steps.groupby(by=['dt_typed']).size()
                if duplicates[duplicates > 1].shape[0] > 0:
                    print('duplicates in step dataframe for {}'.format(both[idx], duplicates[duplicates > 1]))
                # NOTE duplciates in combination can only exist if there are duplciates in the original timestamps
                #print('finished 3rd check')

                df = pd.DataFrame(index=TIME_INDEX)
                df = df.merge(df_sleep[['sleep_value','dt_typed']],how='left',left_index=True,right_on='dt_typed')
                df = df.merge(df_steps[['steps','dt_typed']],how='left',left_on='dt_typed',right_on='dt_typed')

                #YSS
                duplicates = df.groupby(by=['dt_typed']).size()
                if duplicates[duplicates > 1].shape[0] > 0:
                    print('duplicates in combined dataframe for {}'.format(both[idx]))
                    duplicates[duplicates > 1].to_csv('debug{}-combine_sleep_step.csv'.format(both[idx][:-4]))
                #print('finished 4th check')

                df = df.drop_duplicates(subset='dt_typed', keep='first') #YSS why should there be any duplicates?
                df = df.set_index('dt_typed')

                # lac1 only gives steps every 5 minutes, so we interpolate
                if phase == 'lac1':
                    df['steps'] = df['steps'].interpolate()

                df.index.names = ['time']
                df.sort_index(inplace=True)
                df.to_csv(os.path.join(combined_dir, both[idx]))
            except:
                print('exception happened in processing', sys.exc_info()) #YSS
                bad_ids.append(both[idx])
    print(phase, bad_ids)

In [ ]:
# UNCOMMENT TO RUN - uw1/2 both have a few empty participants
# these will be ignored and their PIDs will be printed

saveAllCombined('uw1')
saveAllCombined('uw2')
#YSS no duplicates for UW-I or UW-II
saveAllCombined('lac1')
#YSS no duplicates for CMU-I
saveAllCombined('lac2')
#YSS it seems that data appears twice under '202.csv', '224.csv', '245.csv', '277.csv'

In [ ]:
# calculates the number of minutes that separates two datetimes
def diff_min(dt1,dt2):
    
    DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
    
    td = datetime.strptime(dt1, DATE_FORMAT) - datetime.strptime(dt2, DATE_FORMAT)
    return int(td.total_seconds() / 60)

Once we have all the combined sleep-steps files, we then want to extract the sleep episodes. To do this, we have two parameters: MIN_CONSEC_NON_AWAKE and MAX_CONSEC_AWAKE.

MIN_CONSEC_NON_AWAKE is the number of consecutive minutes that a student has to be asleep/restless for an episode to be recognized, and MAX_CONSEC_AWAKE is the number of consecutive awake minutes that have to be recognized for the termination of an episode. Any timepoint where steps are positive is considered awake.


In [ ]:
MIN_CONSEC_NON_AWAKE = 20
MAX_CONSEC_AWAKE = 5

In [ ]:
def getSubjectSleepEpisodes(phase, 
                            filename,
                            min_consec_non_awake = MIN_CONSEC_NON_AWAKE,
                            max_consec_awake = MAX_CONSEC_AWAKE):
        
    path = getPath(phase)

    COMBINED_DIR = os.path.join(path, 'sleep_steps_data/')
    EPISODES_DIR = os.path.join(path, 'sleep_episodes/')
    
    # Read combined data
    filename = os.path.join(COMBINED_DIR, filename)
    print('episodes for', filename) #YSS
    combined_df = pd.read_csv(filename, index_col=0)
    
    #YSS
    duplicates = combined_df.groupby(combined_df.index).size()
    if duplicates[duplicates > 1].shape[0] > 0:
        print('duplicates in combined dataframe for {}'.format(filename))
        duplicates[duplicates > 1].to_csv('debug{}-episode_sleep_step.csv'.formt(filename[:-4]))
    
    # Define fields
    columns = ['start_time', 'end_time', 'length', 'time_asleep', 'time_restless', 'time_awake']
    
    # Make dictionary
    episode_dict = {} # start_time -> ['end_time', 'length', 'time_asleep', 'time_restless', 'time_awake']
    
    # tracks the last time point the subject was awake
    last_awake = combined_df.index[0]
    
    # used for tracking when in or not in and episode
    consec_non_awake = 0
    consec_awake = 0
    on_episode = False
    start_index = None
    combined_df = combined_df[~combined_df.index.duplicated(keep='first')] #YSS what is this about duplicated   
    
    # iterate through the subject's combined dataframe
    for index, row in combined_df.iterrows():
        sleep = row['sleep_value']

        # update counts
        if math.isnan(sleep) or sleep == 3 or sleep == 0: # 3 = awake, 0 = missing in lac1
            consec_awake += 1
            consec_non_awake = 0
            last_awake = index
        elif sleep == 1: # 1 = asleep
            consec_awake = 0
            consec_non_awake += 1
        elif sleep == 2: # 2 = restless
            consec_awake = 0
            consec_non_awake += 1
        else:
            raise Exception('sleep value is not NaN, 1, 2, or 3')

        # update episode status
        # starting new episode
        if not on_episode and consec_non_awake >= min_consec_non_awake:
            on_episode = True
            start_index = last_awake
            start_loc = combined_df.index.get_loc(index)
        # ending episode
        elif on_episode and consec_awake >= max_consec_awake:
            index_loc = combined_df.index.get_loc(index)
            end_index = combined_df.index[index_loc-max_consec_awake+1]
            length = diff_min(end_index,start_index)
            temp_df = combined_df.iloc[start_loc:index_loc]
            time_awake = len(temp_df[temp_df['sleep_value']==3])
            time_restless = len(temp_df[temp_df['sleep_value']==2])
            time_asleep = length - time_restless - time_awake
            episode_dict[start_index] = [end_index, length, time_asleep, time_restless, time_awake]
            start_index = index
            on_episode = False

    # Make dataframe from dictionary
    episode_df = pd.DataFrame.from_dict(episode_dict, orient='index', columns=columns[1:])
    episode_df.index.name = columns[0]
    
    return episode_df

In [ ]:
#MIN_CONSEC_NON_AWAKE = 4
#MAX_CONSEC_AWAKE = 2
#combined_df.loc[combined_df.iloc[:9].index, 'sleep_value'] = pd.Series([1, 1, 3, 1, 1, 1, 3, 1, 1], index=combined_df.iloc[:9].index) # no episode
#combined_df.loc[combined_df.iloc[:9].index, 'sleep_value'] = pd.Series([3, 3, 1, 1, 1, 1, 3, 1, 1], index=combined_df.iloc[:9].index) # no episode
#combined_df.loc[combined_df.iloc[:9].index, 'sleep_value'] = pd.Series([1, 3, 1, 1, 1, 1, 3, 3, 1], index=combined_df.iloc[:9].index)
#combined_df.loc[combined_df.iloc[:9].index, 'sleep_value'] = pd.Series([3, 1, 1, 1, 1, 3, 1, 3, 3], index=combined_df.iloc[:9].index)
#YSS issues in episode calculations (all minor):
# - sensitive to awake oscillation at the start of an episode but not in the middle
# - neither start_time nor end_time are inclusive 
#   (the episode actually starts after start_time and ends before end_time)
# - actual length of time is smaller by one minute
# - cannot find episodes if there are not enough awake minutes at the end of the timeseries; 
#   can be fixed by padding MAX_CONSEC_AWAKE awake minutes at the end

In [ ]:
def computeAllEpisodesForParams(phase,
                                min_consec_non_awake, 
                                max_consec_awake):

    path = getPath(phase)

    COMBINED_DIR = os.path.join(path, 'sleep_steps_data/')
    EPISODES_DIR = os.path.join(path, 'sleep_episodes/')    
        
    if not os.path.exists(EPISODES_DIR): 
        os.makedirs(EPISODES_DIR) 
    
    for src in tqdm(os.listdir(COMBINED_DIR)):

        dest = 'EPI_' + src.split('.')[0] + '.csv'

        dest = os.path.join(EPISODES_DIR,dest)

        if not os.path.exists(dest):    # remove to writeover existing files
            df = getSubjectSleepEpisodes(
                     phase,
                     src,
                     min_consec_non_awake,
                     max_consec_awake)
            df.to_csv(dest)
        

In [ ]:
# UNCOMMENT TO RUN - took 3-5 hours on my personal computer 
# for phase in ['uw1', 'uw2', 'lac1', 'lac2', 'nh']:
#     computeAllEpisodesForParams(phase,
#                             MIN_CONSEC_NON_AWAKE,
#                             MAX_CONSEC_AWAKE)
computeAllEpisodesForParams('uw1', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE) #YSS
computeAllEpisodesForParams('uw2', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE) #YSS
computeAllEpisodesForParams('lac1', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE) #YSS
computeAllEpisodesForParams('lac2', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE) #YSS
#computeAllEpisodesForParams('nh', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE) #YSS

In [ ]:
computeAllEpisodesForParams('uw2', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE)

In [ ]:
computeAllEpisodesForParams('lac1', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE)

In [ ]:
computeAllEpisodesForParams('lac2', MIN_CONSEC_NON_AWAKE, MAX_CONSEC_AWAKE)

Once we've extracted the sleep episodes (which are in the sleep_episodes/ folder under a subdirectory titled "MIN_CONSEC_NONAWAKE""MAX_CONSEC_AWAKE"), we then have to compute the sleep features of interest.

To do so, we need to convert the bedtime and waketime into minutes after a given zero (e.g. minutes after 6 pm) since time is modulo 24 hr. For bedtime, students generally go to bed after 6 pm, and for waketime, students generally wake up after 4 am (these numbers were not chosen formally). Thus, we set BED_ZERO = 18 (i.e. 6 pm), and WAKE_ZERO = 4 (i.e. 4 am), and convert bedtime and waketime to the number of minutes after these respective zeros.

Additionally, we identified the main sleep episode of Day n as the longest sleep episode which started between noon of Day n and noon of Day (n+1).

In [19]:
BED_ZERO = 18
WAKE_ZERO = 4

In [ ]:
# returns the main episode date for given time
def getWindow(time):
    if time.hour < 12:
        time = time - timedelta(days=1)
    return time.date()

In [ ]:
def getEpisodeData(phase,
                   filename): # ex: 'PID001.csv'
    
    DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
    
    path = getPath(phase)
    
    EPISODES_DIR = os.path.join(path, 'sleep_episodes/')
    file = os.path.join(EPISODES_DIR, 'EPI_'+ filename.split('.')[0] + '.csv')
    
    df = pd.read_csv(file, header=0) #YSS consider using pd.read_csv(file, header=0, parse_dates=['start_time', 'end_time'])
    
    df['start_time'] = df['start_time'].apply(lambda x: datetime.strptime(x,DATE_FORMAT)) #YSS not needed if using parse_dates in read_csv
    df['end_time'] = df['end_time'].apply(lambda x: datetime.strptime(x,DATE_FORMAT)) #YSS not needed if using parse_dates in read_csv
    
    # define a main sleep episode as the longest sleep episode
    # in sleep window
    df['subject_id'] = filename.split('.')[0]
    df['main_episode_of'] = df['start_time'].apply(lambda x: getWindow(x))
    #df = df.sort_values(by='length', ascending=False) #YSS not needed
    #df = df.sort_values(by='main_episode_of') #YSS not needed
    
    #YSS
    df.index.name = 'inds'
    main_index = df.reset_index().groupby(by=['main_episode_of']).apply(lambda x: 
                                                                        x.sort_values(by=['length'], 
                                                                                      ascending=False).iloc[0]['ind'])
    df['main_sleep'] = False
    df.loc[df.index.isin(main_index), 'main_sleep'] = True
    
    return df

In [ ]:
def getMainEpisodeData(phase, filename):
    
    df = getEpisodeData(phase, filename)
    df = df.sort_values(by='length', ascending=False)
    df = df.drop_duplicates(subset='main_episode_of',keep='first')
    df = df.sort_values(by='main_episode_of') 
    
    return df

In [18]:
def timeToMin(time, zero_hour):
    
    hour = time.hour
    minute = time.minute
    
    if hour < zero_hour: #YSS so if bed_time is 5pm with BED_ZERO as 6pm you want to return 
        hour += 24
        
    return (hour-zero_hour)*60 + minute

Here, we compute various sleep features of interest for a given time window.

MSSD refers to mean successive squared difference. This is a measure of variability that also takes into account the temporal nature of the data. For instance, the MSSD of [3,4,10] is ((4-3)^2 + (10-4)^2)/2 = 37/2

In [ ]:
def getMainEpisodeMSSD(main_episode_df):
    
    main_episode_df = main_episode_df.sort_values(by='main_episode_of')
    
    main_episode_df['bedtime'] = main_episode_df['start_time'].apply(lambda x: timeToMin(x,BED_ZERO))
    main_episode_df['waketime'] = main_episode_df['end_time'].apply(lambda x: timeToMin(x,WAKE_ZERO))
    main_episode_df['midpoint_sleep'] = (main_episode_df['waketime'] + main_episode_df['bedtime']) / 2.0

    
    count = 0
    bt_total = []
    wt_total = []
    mp_total = []
    
    for i, idx1 in enumerate(main_episode_df.index): #YSS this does not handle data dicontinuity (e.g. missing days)
        row1 = main_episode_df.loc[idx1]
        
        if i < len(main_episode_df.index)-1:
            idx2 = main_episode_df.index[i+1]
            row2 = main_episode_df.loc[idx2]
        else:
            row2 = None
        
        if row2 is not None:
            count += 1
            bt_total.append((row2['bedtime']-row1['bedtime'])**2)
            wt_total.append((row2['waketime']-row1['waketime'])**2)
            mp_total.append((row2['midpoint_sleep']-row1['midpoint_sleep'])**2)
        
    
    if count == 0:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    bt_mssd = np.array(bt_total).mean() / (float(count) * 3600) #YR: why deviding by float(count) SP: there should not be
    wt_mssd = np.array(wt_total).mean() / (float(count) * 3600)
    mp_mssd = np.array(mp_total).mean() / (float(count) * 3600)
        
    bt_mssd_median = median(bt_total)
    wt_mssd_median = median(wt_total)
    mp_mssd_median = median(mp_total)
        
    return bt_mssd, wt_mssd, mp_mssd, bt_mssd_median, wt_mssd_median, mp_mssd_median

For a given time window, computeSummaryStats computes all the sleep features for a single participant, while computeAllSummaryStats computes all the sleep features for **all** participants for the given time window.

In [ ]:
# start_window and end_window are date objects
# extracts start to end inclusive
def computeSummaryStats(phase, 
                        filename, # ex: 'PID001.csv'
                        start_window, 
                        end_window):
    
        
        DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
        
        summary = dict()
        
        subject_id = filename.split('.')[0]
        
        summary['subject_id'] = subject_id
        
 
        all_df = getEpisodeData(phase, subject_id)
        main_df = getMainEpisodeData(phase, subject_id)

        start = pd.Timestamp(start_window)
        end = pd.Timestamp(end_window)
        main_df = main_df[main_df['main_episode_of'].between(start,end)]
        
        #all_df['start_time'] = all_df['start_time'].apply(lambda x: datetime.strptime(str(x),DATE_FORMAT)) #YSS this is already done in getEpisodeData
        #all_df['main_episode_of'] = all_df['start_time'].apply(lambda x: getWindow(x)) #YSS this s already done in getEpisodeData
        all_df = all_df[all_df['main_episode_of'].between(start,end)] 
        
        nap_df = all_df[~all_df.start_time.isin(main_df.start_time)]
        
        #YSS
        if all_df.shape[0] != (main_df.shape[0] + nap_df.shape[0]):
            print('investigate construction of nap_df in {} (inequality)'.format(filename))
        if all_df[~all_df['main_leep']].shape[0] != nap_df.shape[0]:
            print('investigate construction of nap_df in {} (alternative construction)'.format(filename))
        
        summary['num_naps'] = len(nap_df)
        summary['avg_nap_length'] = nap_df['length'].mean()
        
        #YSS
        if all_df.shape[0] > 0:
            if all_df[~all_df['main_leep']]['length'].mean() != summary['avg_nap_length']:
                print('investigate construction of nap_df {} (alternative construction - mean)'.format(filename))
            if all_df[~all_df['main_leep']]['length'].sum() != nap_df['length'].sum():
                print('investigate construction of nap_df {} (alternative construction - sum)'.format(filename))
            if ((all_df.groupby(all_df.set_index('start_time').index.dayofyear)['length'].sum()).mean() - 
                all_df['length'].sum() / len(all_df['main_episode_of'].unique())).abs() > 0.01:
                print('investigate construction of nap_df {} (alternative construction - sum)'.format(filename))
        
        try:
            summary['frac_napped_of_total_sleep'] = nap_df['length'].sum() / all_df['length'].sum()
            summary['avg_24_hr_sleep'] = all_df['length'].sum() / len(all_df['main_episode_of'].unique()) #YSS issue: all_df should first be filtered on days with main_sleep; the alternative is to groupby dayofyear, sum on each day, and the get the mean of the result
            summary['frac_sleep_episodes_as_naps'] = len(nap_df) / len(all_df)
            
        except: # all_df is empty
            #YSS suggest filling in all the other feilds and returning summary
            summary['frac_napped_of_total_sleep'] = np.nan
            summary['avg_24_hr_sleep'] = np.nan
            summary['frac_sleep_episodes_as_naps'] = np.nan
        
        summary['frac_nights_with_data'] = len(main_df) / ((end_window-start_window).days + 1)
 

        if len(main_df) == 0:
            #YSS this is not consistent with try-catch above
            return None

        
        main_df['bedtime'] = main_df['start_time'].apply(lambda x: timeToMin(x,BED_ZERO)) #YSS I do not think it matters to change the reference point
        main_df['waketime'] = main_df['end_time'].apply(lambda x: timeToMin(x,WAKE_ZERO)) #YSS I do not think it matters to change the reference point
        main_df['midpoint_sleep'] = (main_df['waketime'] + main_df['bedtime']) / 2.0 #YSS waketime and bedtime are calculated wrt different reference points; what does it mean to find their average? what reference point to use to interpret this average?
        #YSS alterantive approain find mid_time based on 'start_time' and length of the main episode then similarly remap it wrt a reference (e.g. BED_ZERO)

            
        main_df['proportion_awake'] = main_df['time_awake'] / main_df['length']
        main_df['proportion_restless'] = main_df['time_restless'] / main_df['length']
        main_df['weekday'] = main_df['main_episode_of'].apply(lambda x: x.weekday())
        
        # 4 = friday night, 5 = saturday night
        main_df['is_weekend'] = main_df['weekday'].apply(lambda x: x in [4,5])
        
        # bedtime measures
        summary['bedtime'] = main_df['bedtime'].mean()
        summary['bedtime_std'] = main_df['bedtime'].std()
    
        # waketime measures
        summary['waketime'] = main_df['waketime'].mean()
        
        # midpoint sleep measures
        summary['midpoint_sleep'] = main_df['midpoint_sleep'].mean()

        # weekday measures
        try:  
            summary['bedtime_weekday'] = main_df.groupby('is_weekend')['bedtime'].mean()[False]        
            summary['waketime_weekday'] = main_df.groupby('is_weekend')['waketime'].mean()[False]
            summary['midpoint_sleep_weekday'] = main_df.groupby('is_weekend')['midpoint_sleep'].mean()[False]            
        except KeyError: #YSS if an empty summary (all fields are np.nan) is returned in the first try-catch this won't be needed
            summary['bedtime_weekday'] = np.nan     
            summary['waketime_weekday'] = np.nan
            summary['midpoint_sleep_weekday'] = np.nan                
        
        # weekend measures
        try:
            summary['bedtime_weekend'] = main_df.groupby('is_weekend')['bedtime'].mean()[True]  
            summary['waketime_weekend'] = main_df.groupby('is_weekend')['waketime'].mean()[True]      
            summary['midpoint_sleep_weekend'] = main_df.groupby('is_weekend')['midpoint_sleep'].mean()[True]      
        except KeyError: #YSS if an empty summary (all fields are np.nan) is returned in the first try-catch this won't be needed
            summary['bedtime_weekend'] = np.nan
            summary['waketime_weekend'] = np.nan
            summary['midpoint_sleep_weekend'] = np.nan

            
        try:
            summary['social_jetlag'] = summary['bedtime_weekend'] - summary['bedtime_weekday'] #YSS this is incorrect midpoint sleep is used for calculating social jetlag
        except:
            summary['social_jetlag'] = np.nan
            
        summary['time_in_bed'] = main_df['length'].mean()
                
        bt_mssd, wt_mssd, mp_mssd, bt_mssd_median, wt_mssd_median, mp_mssd_median = getMainEpisodeMSSD(main_df)
        
        summary['bedtime_mssd'] = bt_mssd
        summary['waketime_mssd'] = wt_mssd
        summary['midpoint_sleep_mssd'] = mp_mssd
        
        summary['bedtime_mssd_median'] = bt_mssd_median
        summary['waketime_mssd_median'] = wt_mssd_median
        summary['midpoint_sleep_mssd_median'] = mp_mssd_median
        
        summary['WASO_fraction'] = main_df['proportion_awake'].mean() #YSS what is WASO?
        summary['restless_fraction'] = main_df['proportion_restless'].mean()
        summary['TST'] = main_df['time_asleep'].mean()
        summary['TST_std'] = main_df['time_asleep'].std()
        
        
        return summary

In [15]:
file = '/Users/yasaman/UWEXP/cmu-sleep-gpa/Fitbit UWEXPI/sleep_episodes/EPI_PID155.csv'
df = pd.read_csv(file, header=0, parse_dates=['start_time', 'end_time'])
df.tail(10)

,start_time,end_time,length,time_asleep,time_restless,time_awake
154,2018-06-03 02:19:00,2018-06-03 08:39:00,380,364,16,0
155,2018-06-03 12:47:00,2018-06-03 21:02:00,495,465,30,0
156,2018-06-04 11:54:00,2018-06-04 19:34:00,460,425,35,0
157,2018-06-05 11:21:00,2018-06-05 19:49:00,508,480,26,2
158,2018-06-06 15:17:00,2018-06-06 19:58:00,281,264,17,0
159,2018-06-07 10:20:00,2018-06-07 14:29:00,249,223,24,2
160,2018-06-08 11:08:00,2018-06-08 17:50:00,402,377,22,3
161,2018-06-09 18:53:00,2018-06-09 20:34:00,101,96,5,0
162,2018-06-11 02:04:00,2018-06-11 10:10:00,486,462,24,0
163,2018-06-12 02:17:00,2018-06-12 10:42:00,505,475,30,0


In [21]:
timeToMin(df['start_time'].iloc[158], BED_ZERO)

1277

In [23]:
btime = datetime.strptime('2018-06-11 22:04:00', '%Y-%m-%d %H:%M:%S')
wtime = datetime.strptime('2018-06-12 06:04:00', '%Y-%m-%d %H:%M:%S')
mtime = datetime.strptime('2018-06-12 02:04:00', '%Y-%m-%d %H:%M:%S')

bedtime = timeToMin(btime, BED_ZERO)
waketime = timeToMin(wtime, WAKE_ZERO)
midpoint = (bedtime + waketime) / 2.0
print(bedtime, waketime, midpoint, end=', ')

244 124 184.0, 

In [24]:
timeToMin(mtime, BED_ZERO)

484

In [25]:
timeToMin(mtime, WAKE_ZERO)

1324

In [26]:
timeToMin(mtime, 0)

124

In [ ]:
# start_window and end_window are date objects
def computeAllSummaryStats(phase,
                           start_window, 
                           end_window,
                           overwrite=False):
    
    path = getPath(phase)

    FEATURES_FOLDER = os.path.join(path, 'computed_features/')
    EPISODES_DIR = os.path.join(path, 'sleep_episodes/')
    feature_file = str(start_window) + '_' + str(end_window) + '.csv'       
    feature_file = os.path.join(FEATURES_FOLDER,feature_file)    
   
    if os.path.exists(feature_file) and not overwrite:
        return 'Already written'
        
    ids = os.listdir(os.path.join(path, 'sleep_steps_data/'))

    
    with open(feature_file, 'w') as f:
    
        header = ['subject_id',
                  'num_naps',
                  'avg_nap_length',
                  'frac_napped_of_total_sleep',
                  'avg_24_hr_sleep',
                  'frac_sleep_episodes_as_naps',
                  'frac_nights_with_data',
                  'bedtime',
                  'waketime',
                  'midpoint_sleep',
                  'time_in_bed',
                  'bedtime_mssd',
                  'bedtime_mssd_median',
                  'bedtime_std',
                  'bedtime_weekend',
                  'bedtime_weekday',
                  'waketime_mssd',
                  'waketime_mssd_median',
                  'waketime_weekend',
                  'waketime_weekday',
                  'midpoint_sleep_mssd',
                  'midpoint_sleep_mssd_median',
                  'midpoint_sleep_weekend',
                  'midpoint_sleep_weekday',
                  'WASO_fraction',
                  'restless_fraction',
                  'social_jetlag',
                  'TST',
                  'TST_std']
        
        w = csv.DictWriter(f, header)
        w.writeheader()
        
        for filename in ids:
    
            summary = computeSummaryStats(phase, 
                                          filename,
                                          start_window, 
                                          end_window)
                
            if not summary:
                summary = dict()
                for val in header:
                    summary[val] = np.nan
                summary['subject_id'] = filename.split('.')[0]
                summary['frac_nights_with_data'] = 0
                
            w.writerow(summary)

**Regression Analyses**

In [ ]:
# UNCOMMENT TO RUN
# for d in ['uw1','uw2','lac1','lac2', 'nh']:
#     new_d = os.path.join(getPath(d),'computed_features/')
#     if not os.path.exists(new_d): 
#         os.makedirs(new_d)

In [ ]:
REGRESSION_PERIODS = [('uw1',date(2018,4,1),date(2018,4,28)),
                      ('uw2',date(2019,4,7),date(2019,5,4)),
                      ('lac1',date(2017,2,7),date(2017,3,7)),
                      ('lac2',date(2018,2,7),date(2018,3,7)),
                      ('nh', date(2016,2,4), date(2016,3,4))]

PREDICTORS = ['bedtime_mssd',
              'TST',
              'midpoint_sleep']

In [ ]:
def computeAllFeatureWindows():
    for period in REGRESSION_PERIODS:

        phase = period[0]
        start_window = period[1]
        end_window = period[2]

        computeAllSummaryStats(phase, start_window, end_window, overwrite=True)    

In [ ]:
def getSummaryStats(phase, start, end):

    start_window = start
    end_window = end
    
    path = getPath(phase)
            
    # get entire semester participant data   
    filename = str(start_window) + '_' + str(end_window) + '.csv'
    filename = os.path.join(path, 'computed_features/', filename)
    summary_stats_df = pd.read_csv(filename)
    summary_stats_df.set_index('subject_id', inplace=True)
    
    return summary_stats_df[summary_stats_df['frac_nights_with_data']>=.2]

In [ ]:
# used to extract first-year student label from EMA for lac2
def getLAC2Freshmen():
    phase = 'lac2'
    path = os.path.join('GPA_data/raw_gpa/',
                        phase + '_pre_post_ema.csv')
    df = pd.read_csv(path, index_col='ID', low_memory=False)
    return list(df[df['IRA_YearOfStudy']==1].index)

In [ ]:
def getGPAData(phase):
    path = os.path.join('GPA_data/cleaned_gpa/', 
                        phase + '_freshmen_gpa.csv')
    return pd.read_csv(path, index_col='subject_id')

In [ ]:
# all predictors of interest, GPA features, no nan rows,
# and participants with >=20% fraction of nights data 
def getRegDF(phase, start, end, predictors, control=None, thresh=0.2):
        
        # compute summary statistics
        computeAllSummaryStats(phase, start, end, overwrite=True) # ADD overwrite=True
        stats_df = getSummaryStats(phase, start, end)
        
        # filter by freshmen
        if phase == 'lac2':
            stats_df = stats_df[stats_df.index.isin(getLAC2Freshmen())]

        # filter by completeness threshold
        stats_df = stats_df[stats_df['frac_nights_with_data'] >= thresh]

        # get GPA data
        gpa_df = getGPAData(phase)
        
        # combine summary statistics with GPA data
        combined_df = stats_df.merge(gpa_df, on='subject_id', how='outer')

        # choose only columns of variables of interest
        columns = predictors + ['cum_gpa', 'term_gpa', 'frac_nights_with_data']
                
        if control:
            columns += [control]        
        
        combined_df = combined_df[columns].dropna()
        
        return combined_df[columns]

In [ ]:
def saveRegressionDF():

    dfs = []
    for period in REGRESSION_PERIODS:
        df = getRegDF(period[0], period[1], period[2], PREDICTORS, thresh=0.2)
        df['cohort'] = period[0]
        dfs.append(df)
    all_reg_dfs = pd.concat(dfs)
    all_reg_dfs.to_csv('regression_df.csv')


In [ ]:
# UNCOMMENT TO RUN
# saveRegressionDF()